In [ ]:
#!pip install 'crewai[tools]'
#!pip install ollama

In [ ]:
!pip install crewai crewai_tools langchain==0.2.6 langchain-core==0.2.15 langchain-experimental==0.0.62 langchain-community==0.2.6 langchain_mistralai

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_core

In [ ]:
import os
os.environ["OPENAI_API_BASE"] = "https://api.mistral.ai/v1"
os.environ["OPENAI_API_KEY"] = "xxxxxxxxx"
os.environ["OPENAI_MODEL_NAME"] = 'mistral-large-latest'
os.environ["SERPER_API_KEY"] ="xxxxxx"

from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI
#from langchain.chains import GraphCypherQAChain
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"


In [ ]:
from langchain.agents import Tool
graph = Neo4jGraph(url=os.environ['NEO4J_URI'], username=os.environ['NEO4J_USERNAME'], password=os.environ['NEO4J_PASSWORD'])

graph.refresh_schema()
api_key=os.environ["OPENAI_API_KEY"]
llm = ChatMistralAI(api_key=api_key, model="mistral-large-latest",temperature=0)
neo4jquery = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)
neo4j_query_tool = Tool(
  name="neo4j_query_tool",
  func=neo4jquery.run,
  description="neo4j database that contains data about the project, user provide english question the tool translates it to query and respond with the answer",
)

#neo4j_query_tool.run("count floors")
neo4j_query_tool.run("how many floors are there?")

In [ ]:
# Agent 1: Researcher
from crewai import Agent, Task, Crew
analyst = Agent(
    role="civil engineer",
    goal="get information about materials from data tool you ask the tool the questions in human language",
    tools = [  neo4j_query_tool],
    verbose=True,
    backstory=(
        "As civil engineer, you get your input from the neo4j_query_tool, you ask the questions in human language, for example, if you want to know number of floors you ask : how many floors are there? "
        "you might need to calculate things by yourself, according to the data you get from the tool. you should not invent anything"
    )
)
# Task for Researcher Agent:
analyst_task = Task(
    description=(
        "get the list of materials. the data is about construction project. "
        "need to get the number of walls, their materials and calculate their volumes, calculate volume data for what you find"
    ),
    expected_output=(
        "list of walls, material and volume"
    ),
    agent=analyst,
    async_execution=True
)
application_crew = Crew(
    agents=[analyst],

    tasks=[analyst_task],

    verbose=True
)
result = application_crew.kickoff()
